In [132]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)
# https://wikidocs.net/60572

In [133]:
z=torch.FloatTensor([1,2,3])
# h=F.softmax(z,dim=0)
# print(h)
# print(h.sum())



In [134]:
z=torch.rand(3,5,requires_grad=True)
h=F.softmax(z,dim=1)
print(h)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [135]:
y=torch.randint(5, (3,)).long()
print(y)
# 3행

tensor([0, 2, 1])


In [136]:
y_one_hot=torch.zeros_like(h)
print(y_one_hot)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [137]:
print(y.unsqueeze(1))
y_one_hot.scatter_(1,y.unsqueeze(1),1) #dim, index, source
print(y_one_hot)

tensor([[0],
        [2],
        [1]])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [138]:

cost=(y_one_hot* -torch.log(h)).sum(dim=1).mean()
print(cost)


tensor(1.4689, grad_fn=<MeanBackward0>)


In [139]:
torch.log(F.softmax(z,dim=1))


tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [140]:
torch.log_softmax(z,dim=1)


tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [141]:
cost = (y_one_hot * -torch.log(h)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [142]:
(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()
# cost = (y_one_hot * -torch.log(h)).sum(dim=1).mean() 와 결과 같음

tensor(1.4689, grad_fn=<MeanBackward0>)

In [143]:
# 손실함수를 적을때 간단한 방법
F.nll_loss(F.log_softmax(z,dim=1),y)
# nll-> Negative log Likelihood 약자로 
# 원-핫 벡터를 넣을 필요없이 바로 실제값을 인자로 사용

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [144]:
# F.cross_entropy()는 F.log_softmax()와 F.nll_loss()를 포함하고 있습니다.
# F.cross_entropy는 비용 함수에 소프트맥스 함수까지 
# 포함하고 있음을 기억하고 있어야 구현 시 혼동하지 않습니다.
F.cross_entropy(z,y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [145]:
x_train=[[1,2,1,1],[2,1,3,2],[3,1,3,4],[4,1,5,5],[1,7,5,5],[1,2,5,6],[1,6,6,6],[1,7,7,7]]
y_train=[2,2,2,1,1,1,0,0]
x_train=torch.FloatTensor(x_train)
y_train=torch.LongTensor(y_train)
print(x_train.shape)
print(y_train.size())


torch.Size([8, 4])
torch.Size([8])


In [146]:
y_one_hot=torch.zeros(8,3)
y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
print(y_one_hot)

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


In [147]:
w=torch.zeros((4,3), requires_grad=True)
b=torch.zeros((1,3), requires_grad=True)
# 회귀식이 1행 3열
optimzer=optim.SGD([w,b], lr=0.1)

for epoch in range(1001):
  # 식이 3가지 이므로 dim=1
  hat=F.softmax(x_train.matmul(w)+b, dim=1)
  cost=(y_one_hot * -torch.log(hat)).sum(dim=1).mean()

  optimzer.zero_grad()
  cost.backward()
  optimzer.step()

  if epoch % 100 ==0:
    print('epoch: {}, cost:{}'.format(epoch,cost.item()))
  

epoch: 0, cost:1.0986123085021973
epoch: 100, cost:0.7041996121406555
epoch: 200, cost:0.6229995489120483
epoch: 300, cost:0.5657169222831726
epoch: 400, cost:0.5152913331985474
epoch: 500, cost:0.467661589384079
epoch: 600, cost:0.4212779700756073
epoch: 700, cost:0.37540146708488464
epoch: 800, cost:0.3297656178474426
epoch: 900, cost:0.2850724160671234
epoch: 1000, cost:0.24815461039543152


In [148]:
pred=F.softmax(x_train.matmul(w)+b, dim=1)
print(pred)
pred.argmax(dim=1)

tensor([[1.1909e-04, 7.5725e-03, 9.9231e-01],
        [3.9348e-03, 1.3411e-01, 8.6196e-01],
        [6.9057e-06, 2.6787e-01, 7.3212e-01],
        [3.1690e-05, 7.8606e-01, 2.1390e-01],
        [3.2070e-01, 6.5953e-01, 1.9770e-02],
        [1.6596e-01, 8.3388e-01, 1.6129e-04],
        [6.3023e-01, 3.6949e-01, 2.7694e-04],
        [8.0781e-01, 1.9218e-01, 1.4840e-05]], grad_fn=<SoftmaxBackward0>)


tensor([2, 2, 2, 1, 1, 1, 0, 0])

In [149]:
model=nn.Linear(4,3)
optimzer=optim.SGD(model.parameters(), lr=0.1)

for epoch in range(1001):
  y_hat=model(x_train)
  cost=F.cross_entropy(y_hat, y_train)

  optimzer.zero_grad()
  cost.backward()
  optimzer.step()

  if epoch % 100 ==0:
    print('epoch: {}, cost:{}'.format(epoch,cost.item()))

epoch: 0, cost:1.8495128154754639
epoch: 100, cost:0.6898943781852722
epoch: 200, cost:0.6092584729194641


epoch: 300, cost:0.5512180924415588
epoch: 400, cost:0.5001412034034729
epoch: 500, cost:0.45194703340530396
epoch: 600, cost:0.40505102276802063
epoch: 700, cost:0.35873350501060486
epoch: 800, cost:0.3129115402698517
epoch: 900, cost:0.26952168345451355
epoch: 1000, cost:0.2419215589761734


In [150]:
model(x_train).argmax(dim=1)

tensor([2, 2, 2, 1, 1, 1, 0, 0])

In [151]:
class SoftmaxClassModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear=nn.Linear(4,3)

  def forward(self, x):
    return self.linear(x)

In [152]:
model=SoftmaxClassModel()
optimizer=optim.SGD(model.parameters(), lr=0.1)

for epoch in range(1001):
  y_hat=model(x_train)
  cost=F.cross_entropy(y_hat, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  if epoch%100==0:
    print('Epoch:{} Cost:{}'.format(epoch, cost.item()))
  

Epoch:0 Cost:1.845719814300537
Epoch:100 Cost:0.647149920463562
Epoch:200 Cost:0.5688682794570923
Epoch:300 Cost:0.5156992077827454
Epoch:400 Cost:0.4717271625995636
Epoch:500 Cost:0.4324863851070404
Epoch:600 Cost:0.39587944746017456
Epoch:700 Cost:0.36050671339035034
Epoch:800 Cost:0.32522761821746826
Epoch:900 Cost:0.2892170250415802
Epoch:1000 Cost:0.2540857791900635


In [153]:
model(x_train).argmax(dim=1)

tensor([2, 2, 2, 1, 1, 1, 0, 0])